# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
!pip install cartopy
!pip install geoviews
!pip install pyproj
# Import API key
geoapify_key = "8e1547fcfcba4ce18d560d1719a97e27"
print(geoapify_key)
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,-0.55,95,9,1.34,NZ,1715196002
1,1,port-aux-francais,-49.3500,70.2167,4.89,74,81,7.52,TF,1715196002
2,2,hermanus,-34.4187,19.2345,10.45,78,0,2.41,ZA,1715196002
3,3,el colomo,19.0500,-104.2500,24.99,62,16,4.35,MX,1715196003
4,4,grytviken,-54.2811,-36.5092,-3.45,69,6,3.06,GS,1715196003


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_cities_df = city_data_df[(city_data_df["Humidity"]<= 70) & (city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 20)]

# Drop any rows with null values
cleaned_vacation_cities_df = vacation_cities_df.dropna(subset=['Humidity', 'Max Temp'])

# Display sample data
cleaned_vacation_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,el colomo,19.0500,-104.2500,24.99,62,16,4.35,MX,1715196003
5,5,padre paraiso,-17.0742,-41.4844,24.88,57,1,2.93,BR,1715196003
28,28,albany,42.6001,-73.9662,26.62,63,7,0.45,US,1715195774
31,31,port-vila,-17.7338,168.3219,23.03,63,100,5.53,VU,1715196005
62,62,port mathurin,-19.6833,63.4167,25.57,67,90,12.05,MU,1715196008
...,...,...,...,...,...,...,...,...,...,...
536,536,byford,-32.2167,116.0000,20.48,44,100,5.14,AU,1715196056
539,539,sarina,-21.4333,149.2167,21.26,1,100,4.70,AU,1715196057
545,545,ardestan,33.3761,52.3694,21.10,19,11,3.05,IR,1715196057
561,561,serdar,38.9764,56.2757,21.23,47,0,6.17,TM,1715196059


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_vacation_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,el colomo,MX,19.0500,-104.2500,62,
5,padre paraiso,BR,-17.0742,-41.4844,57,
28,albany,US,42.6001,-73.9662,63,
31,port-vila,VU,-17.7338,168.3219,63,
62,port mathurin,MU,-19.6833,63.4167,67,
...,...,...,...,...,...,...
536,byford,AU,-32.2167,116.0000,44,
539,sarina,AU,-21.4333,149.2167,1,
545,ardestan,IR,33.3761,52.3694,19,
561,serdar,TM,38.9764,56.2757,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
     # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # name_address
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    # print(f"{name_address}")
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
el colomo - nearest hotel: Nelsal Pensao
padre paraiso - nearest hotel: Nelsal Pensao
albany - nearest hotel: Nelsal Pensao
port-vila - nearest hotel: Nelsal Pensao
port mathurin - nearest hotel: Nelsal Pensao
buka - nearest hotel: Nelsal Pensao
tura - nearest hotel: Nelsal Pensao
pacific grove - nearest hotel: Nelsal Pensao
jamestown - nearest hotel: Nelsal Pensao
ar rutbah - nearest hotel: Nelsal Pensao
saint-pierre - nearest hotel: Nelsal Pensao
lompoc - nearest hotel: Nelsal Pensao
maun - nearest hotel: Nelsal Pensao
shakawe - nearest hotel: Nelsal Pensao
cooper's town - nearest hotel: Nelsal Pensao
trail - nearest hotel: Nelsal Pensao
sakakah - nearest hotel: Nelsal Pensao
hirara - nearest hotel: Nelsal Pensao
kununurra - nearest hotel: Nelsal Pensao
okakarara - nearest hotel: Nelsal Pensao
talara - nearest hotel: Nelsal Pensao
albion - nearest hotel: Nelsal Pensao
luena - nearest hotel: Nelsal Pensao
sale - nearest hotel: Nelsal Pensao
rochester - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
3,el colomo,MX,19.0500,-104.2500,62,Nelsal Pensao
5,padre paraiso,BR,-17.0742,-41.4844,57,Nelsal Pensao
28,albany,US,42.6001,-73.9662,63,Nelsal Pensao
31,port-vila,VU,-17.7338,168.3219,63,Nelsal Pensao
62,port mathurin,MU,-19.6833,63.4167,67,Nelsal Pensao
...,...,...,...,...,...,...
536,byford,AU,-32.2167,116.0000,44,Nelsal Pensao
539,sarina,AU,-21.4333,149.2167,1,Nelsal Pensao
545,ardestan,IR,33.3761,52.3694,19,Nelsal Pensao
561,serdar,TM,38.9764,56.2757,47,Nelsal Pensao


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
filtered_hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]
map_plot_2 = filtered_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1.0,
    color="City",
    hover_cols=['Hotel Name', 'City', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)